En el ejercicio de ayer estuvimos viendo el groupby y pudimos observar que los datos estaban un poco sucios... 😭. En los ejercicios de hoy pondremos en práctica los apply, y nos crearemos algunas funciones que nos van a permitir limpiar un poco los datos, para que podamos extraer conclusiones de una forma más sencilla.

Los objetivos de la clase de hoy son:
1. Cread una columna nueva y una función que nos de el mes donde ocurrió el ataque. Tened en cuenta que no todas las filas tienen la misma estructura y que puede que no haya la información de mes. En ese caso devolved un nulo (NaN).
💡 Pista 💡Podéis usar regex.


In [54]:
import pandas as pd
import numpy as np
import re
import random
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [55]:
df = pd.read_csv('attacks_modificado_3.csv', index_col= 0)
df.head(2)

,unnamed:_0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,href,siglo,fatal_(y/n),injury,date
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,6,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,Y,FATAL,03-Jun-2018


In [56]:
# Exploramos la columna date de nuestro df y observamos sus valores estadísticos y valores nulos.

df['date'].value_counts()

No date                                                             6
No date, Before 1963                                                5
Before 1906                                                         4
Before 1958                                                         4
18-Aug-2001                                                         3
24-Jun-2015                                                         3
31-Oct-2003                                                         3
1960s                                                               3
20-Sep-2015                                                         3
31-May-1997                                                         3
1700s                                                               3
20-Apr-2003                                                         3
1941-1945                                                           3
Before 1962                                                         3
09-Jul-1994         

In [57]:
df['date'].describe()

count        1672
unique       1522
top       No date
freq            6
Name: date, dtype: object

In [58]:
df['date'].isnull().sum()

0

In [59]:
# Hacemos una copia del df

df2 = df.copy()

In [60]:
# Hacemos la función para extraer el mes

def sacar_mes(x):
    try:
        return x.split('-')[1]
    except:
        return np.NaN

In [61]:
# Aplicamos la función, creando una columna nueva

df2['month'] = df2['date'].apply(sacar_mes)

In [62]:
# Comprobamos

df2.head(2)

,unnamed:_0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,href,siglo,fatal_(y/n),injury,date,month
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018,Jun
1,6,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,Y,FATAL,03-Jun-2018,Jun


In [63]:
df2['month'].value_counts()

Aug                       156
Sep                       153
Oct                       146
Jul                       143
Jun                       136
Jan                       127
Apr                       124
May                       105
Dec                       104
Mar                       102
Nov                        99
Feb                        96
1945                        4
 1950                       3
1959                        2
1966                        2
1923                        1
1996                        1
1899                        1
1963                        1
1913                        1
1998                        1
1889                        1
1995                        1
1946                        1
1989                        1
1987                        1
1956                        1
1905                        1
1945, more likely 1945      1
1942                        1
28                          1
2009                        1
1961      

In [64]:
# Nos saca valores que no son el mes. Decidimos usar regex

In [65]:
patron = '-(...)-'

In [66]:
# Usaremos este método
re.findall(patron, '12-Apr-2001')[0] # el findall nos saca una lista, por eso le pedimos el indice 0

'Apr'

In [67]:
def sacar_mes_regex(x):
    try:
        return re.findall(patron, x)[0]  
    except:
        return np.NaN

In [68]:
# Aplicamos

df2['month'] = df2['date'].apply(sacar_mes_regex)

In [69]:
# Comprobamos

df2.head(2)

,unnamed:_0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,href,siglo,fatal_(y/n),injury,date,month
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018,Jun
1,6,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,Y,FATAL,03-Jun-2018,Jun


In [70]:
df2['month'].value_counts()

Aug    156
Sep    153
Oct    146
Jul    143
Jun    136
Jan    127
Apr    124
May    105
Dec    104
Mar    102
Nov     99
Feb     96
Name: month, dtype: int64


2. Vamos con la columna fatal_(y/n). Lo primero que tenéis que hacer es evaluar los valores únicos que hay. Esperaríamos tener solo dos valores: y y n, pero tristemente no debería ser así 😭. Cread una columna nueva y una función que devuelva únicamente y o n y que devuelva un nulo (NaN) si no se encuentra el valor.


In [71]:
# Exploramos los valores que hay en la columna que queremos limpiar.

df2['fatal_(y/n)'].unique()

array(['N', 'Y', nan, 'M', '2017', ' N', 'UNKNOWN'], dtype=object)

In [72]:
# Creamos la función.

def devolver_yn(x):
    if x == 'Y':
        return 'Y'
    elif x == 'N':
        return 'N'
    else:
        return np.NaN

In [73]:
# Aplicamos nuestra función, creando una nueva columna.

df2['fatal_y/n'] = df2['fatal_(y/n)'].apply(devolver_yn)

In [74]:
# Comprobación.

df2['fatal_y/n'].unique()

array(['N', 'Y', nan], dtype=object)

3. Seguid la misma lógica que en el ejercicio anterior pero con la columna sex. Pero en este caso vamos a incluir un retito más: En este caso querremos tener dos valores únicos F y M. De nuevo en la columna hay otros carácteres raros como 'M ', 'lli', 'N', '.'. En esos casos, queremos que se reemplacen por M o F de forma aleatoria (tendréis que usar la librería random). En caso de que sea nulo, que la función devuelva nulo (np.nan).


In [75]:
df2['sex_'].unique()

array(['F', 'M', 'lli', nan], dtype=object)

In [76]:
def limpiar_mf(x):
    if x == 'F':
        return 'F'
    elif x == 'M':
        return 'M'
    else:
        lista =['M', 'F']
        return random.choice(lista)

In [77]:
df2['sex_m_f'] = df2['sex_'].apply(limpiar_mf)

In [78]:
# Comprobamos.

df2['sex_m_f'].unique()

array(['F', 'M'], dtype=object)

In [79]:
df2.head(2)

,unnamed:_0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,href,siglo,fatal_(y/n),injury,date,month,fatal_y/n,sex_m_f
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018,Jun,N,F
1,6,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,Y,FATAL,03-Jun-2018,Jun,Y,M


4. Una vez que hayáis terminado eliminad las columnas originales conservando las nuevas.



In [80]:
df2.drop(columns = ['sex_', 'fatal_(y/n)'], axis= 1, inplace=True)

In [81]:
df2.head(2)

,unnamed:_0,case_number,year,type,country,area,location,activity,name,age,time,species_,href,siglo,injury,date,month,fatal_y/n,sex_m_f
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018,Jun,N,F
1,6,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,18,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,FATAL,03-Jun-2018,Jun,Y,M


5. Guardad el dataframe en un csv que usaremos más adelante

In [82]:
df2.to_csv('attacks_modificado_4.csv')